In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv('/users/jjong/desktop/vscode/src/Hotel_Reviews.csv')
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [7]:
data['Hotel_Address'].nunique()

1493

In [10]:
df = data.drop(['lat', 'lng'], axis=1)

In [16]:
x1 = df['Negative_Review'].values
y1 = df['Review_Total_Negative_Word_Counts'].values
x2 = df['Positive_Review'].values
y2 = df['Review_Total_Positive_Word_Counts'].values

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x1)
tokenizer.fit_on_texts(x2)

In [19]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'was': 3,
 'to': 4,
 'a': 5,
 'room': 6,
 'in': 7,
 'very': 8,
 'no': 9,
 'staff': 10,
 'of': 11,
 'for': 12,
 'location': 13,
 'hotel': 14,
 'is': 15,
 'i': 16,
 'we': 17,
 'it': 18,
 'were': 19,
 'not': 20,
 'breakfast': 21,
 'good': 22,
 'negative': 23,
 'with': 24,
 'great': 25,
 'but': 26,
 'on': 27,
 'at': 28,
 'friendly': 29,
 't': 30,
 'had': 31,
 'from': 32,
 'helpful': 33,
 'bed': 34,
 'nice': 35,
 'that': 36,
 'clean': 37,
 'rooms': 38,
 'have': 39,
 'you': 40,
 'as': 41,
 'there': 42,
 'this': 43,
 'so': 44,
 'our': 45,
 'comfortable': 46,
 'excellent': 47,
 'all': 48,
 'my': 49,
 'are': 50,
 'small': 51,
 'they': 52,
 'be': 53,
 'would': 54,
 'stay': 55,
 'nothing': 56,
 'service': 57,
 'really': 58,
 'bathroom': 59,
 'which': 60,
 'out': 61,
 'could': 62,
 'when': 63,
 'one': 64,
 'too': 65,
 'us': 66,
 'bar': 67,
 'only': 68,
 'lovely': 69,
 'an': 70,
 'everything': 71,
 'positive': 72,
 'close': 73,
 's': 74,
 'also': 75,
 'shower': 76,
 'like': 7

In [20]:
x1_tk = tokenizer.texts_to_sequences(x1)
x2_tk = tokenizer.texts_to_sequences(x2)

In [21]:
max_len1 = max([len(s) for s in x1_tk])
max_len2 = max([len(s) for s in x2_tk])

In [26]:
x1_sq = pad_sequences(x1_tk, maxlen=max_len1, padding='pre')
x2_sq = pad_sequences(x2_tk, maxlen=max_len2, padding='pre')
x2_sq.shape

(515738, 393)

In [27]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

81295

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x1_sq, y1, test_size=0.2, shuffle=True, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(412590, 406) (412590,)
(103148, 406) (103148,)


In [28]:
from tensorflow.keras import layers
from tensorflow.keras import Model

# Body
inputs = layers.Input(shape=(max_len1,))
x = layers.Embedding(vocab_size, 64, embeddings_regularizer='l2')(inputs)
x = layers.GRU(units=64, return_sequences=True)(x)
x = layers.GRU(units=64, return_sequences=False)(x)

# header
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(1, activation='linear')(x)  # activation='linear' 생략 가능  

model = Model(inputs=inputs, outputs=outputs)    
model.compile(optimizer='adam', loss='mse')  
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 406)]             0         
                                                                 
 embedding (Embedding)       (None, 406, 64)           5202880   
                                                                 
 gru (GRU)                   (None, 406, 64)           24960     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5254913 (20.05 MB)
Trainable params: 5254913 (2

In [32]:
from tensorflow.keras.callbacks import EarlyStopping

es_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, batch_size=64, epochs=300, validation_split=0.2, callbacks=[es_callback])

Epoch 1/300
  43/5158 [..............................] - ETA: 24:16 - loss: 1091.4646

KeyboardInterrupt: 